In [440]:
import sys
print("Python Version:", sys.version)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import seaborn as sns
pd.set_option('display.max_rows', 500)
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

Python Version: 3.7.6 (default, Jan  8 2020, 20:23:39) [MSC v.1916 64 bit (AMD64)]


In [441]:
# yymmdd format
dates = ['190803',
        '190810',
        '190817',
        '190824',
        '190831',
        '190907',
        '190914',
        '190921',
        '190928']

# iterate through dates and pull in csv and cat dataframes together
df_turns = []
for date in dates:
    df_turns.append(pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_'+date+'.txt'))
    
# Concat all dataframes
df_turns = pd.concat(df_turns)  

In [442]:
#Clean up the column Names
new_col_names = [name.strip() for name in df_turns.columns]
df_turns.columns = new_col_names


In [443]:
# Remove audited rows
df_turns=df_turns[df_turns.DESC!='RECOVR AUD']
# remove column?
del df_turns['DESC']

In [444]:
# Create a DateTime column that can be used for timeseries
df_turns.loc[:,'DATETIME'] = pd.to_datetime(df_turns['DATE'] + ' ' + df_turns['TIME'])

#Convert Date column to proper date type.
df_turns['DATE']  = pd.to_datetime(df_turns['DATE'],format='%m/%d/%Y')

#convert time to datetime type
df_turns['TIME']=pd.to_datetime(df_turns['TIME'], format='%H:%M:%S')

In [445]:
#backup
df_backup=df_turns.copy()

In [446]:
#Sort in order of datetime
df_turns = df_turns.sort_values(by=['DATETIME']).reset_index()

In [447]:
# Caluclate Entries Aggregated at station level by DateTime
df_turns['Entry_Diff']=df_turns.groupby(['STATION', 'C/A', 'UNIT', 'SCP'],as_index=False)['ENTRIES'].transform(pd.Series.diff)['ENTRIES']

# Caluclate Exits Aggregated at station level by DateTime
df_turns['Exit_Diff']=df_turns.groupby(['STATION', 'C/A', 'UNIT', 'SCP'],as_index=False)['EXITS'].transform(pd.Series.diff)['EXITS']



In [448]:
#Absolute Value to deal with counting backward issues 
df_turns['Entry_Diff'] = abs(df_turns['Entry_Diff'])
df_turns['Exit_Diff']=abs(df_turns['Exit_Diff'])

# Calculate both
df_turns['Total_Traffic']=df_turns['Entry_Diff']+df_turns['Exit_Diff']

### Rami's Filtering

In [449]:
def Add_Weekday(data_frame, column='Date'):
    dmap = {0:'Mon', 1: 'Tue', 2: 'Wed', 3:'Thu', 4:'Fri', 5:'Sat', 6:'Sun'}
    #Convert column to date
    data_frame[column] = pd.to_datetime(data_frame[column])
    #Add column which shows the Weekday in integer
    data_frame['Day_Number'] = data_frame[column].apply(lambda x: x.dayofweek)
    #Add column which shows the Weekday in words
    data_frame['Weekday'] = data_frame['Day_Number'].map(dmap)
    return data_frame
df_turns_backup2 = df_turns.copy()

In [450]:
df_turns = Add_Weekday(df_turns, 'DATE')

In [451]:
#Filter out the Null Values
df_turns.dropna(subset=["Entry_Diff","Exit_Diff",'Total_Traffic'], axis=0, inplace=True)
df_turns.head()

,index,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,ENTRIES,EXITS,DATETIME,Entry_Diff,Exit_Diff,Total_Traffic,Day_Number,Weekday
2554,118307,PTH03,R552,00-01-08,JOURNAL SQUARE,1,PTH,2019-07-27,1900-01-01 00:24:57,65,30,2019-07-27 00:24:57,2.0,0.0,2.0,5,Sat
2592,118308,PTH03,R552,00-01-08,JOURNAL SQUARE,1,PTH,2019-07-27,1900-01-01 00:38:57,65,30,2019-07-27 00:38:57,0.0,0.0,0.0,5,Sat
2636,118309,PTH03,R552,00-01-08,JOURNAL SQUARE,1,PTH,2019-07-27,1900-01-01 00:52:57,65,30,2019-07-27 00:52:57,0.0,0.0,0.0,5,Sat
2640,118310,PTH03,R552,00-01-08,JOURNAL SQUARE,1,PTH,2019-07-27,1900-01-01 00:56:25,4043,3513,2019-07-27 00:56:25,3978.0,3483.0,7461.0,5,Sat
4489,118311,PTH03,R552,00-01-08,JOURNAL SQUARE,1,PTH,2019-07-27,1900-01-01 01:06:57,65,30,2019-07-27 01:06:57,3978.0,3483.0,7461.0,5,Sat


In [452]:
dfc_grouped = df_turns.groupby(['DATE', 'STATION', 'Weekday'])['Entry_Diff', 'Exit_Diff', 'Total_Traffic'].sum().reset_index()

C:\Users\atag3\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


In [453]:
#The upper_quant_df outputs a dataframe of the upper 95% values for each station
upper_quant_df = dfc_grouped.groupby(['STATION'])['Total_Traffic'].quantile([.95]).reset_index()
#Then I merge it with the grouped data 
df_turns_no_outlier = dfc_grouped.merge(upper_quant_df, on='STATION')
df_turns_no_outlier.head()

,DATE,STATION,Weekday,Entry_Diff,Exit_Diff,Total_Traffic_x,level_1,Total_Traffic_y
0,2019-07-27,1 AV,Sat,4308.0,5146.0,9454.0,0.95,40228.0
1,2019-07-28,1 AV,Sun,4106.0,5309.0,9415.0,0.95,40228.0
2,2019-07-29,1 AV,Mon,14483.0,16525.0,31008.0,0.95,40228.0
3,2019-07-30,1 AV,Tue,16375.0,18206.0,34581.0,0.95,40228.0
4,2019-07-31,1 AV,Wed,16812.0,19777.0,36589.0,0.95,40228.0


In [454]:
#delete the rows where The difference between Total_Traffic_y and Total_Traffic_x < 0
df_turns_no_outlier['Diff'] = df_turns_no_outlier['Total_Traffic_y'] - df_turns_no_outlier['Total_Traffic_x']
df_turns_no_outlier = df_turns_no_outlier[df_turns_no_outlier['Diff']>= 0]

In [455]:
df_turns_no_outlier.sample(50)

,DATE,STATION,Weekday,Entry_Diff,Exit_Diff,Total_Traffic_x,level_1,Total_Traffic_y,Diff
16450,2019-08-13,KINGS HWY,Tue,25752.0,23590.0,49342.0,0.95,60002.40,10660.40
2415,2019-08-17,2 AV,Sat,13847.0,13309.0,27156.0,0.95,36909.10,9753.10
23755,2019-08-30,SUTTER AV-RUTLD,Fri,6364.0,7163.0,13527.0,0.95,16007.00,2480.00
2660,2019-08-10,21 ST-QNSBRIDGE,Sat,8043.0,7720.0,15763.0,0.95,20378.00,4615.00
3399,2019-09-25,3 AV,Wed,5528.0,5971.0,11499.0,0.95,11999.40,500.40
1834,2019-08-03,175 ST,Sat,8488.0,5434.0,13922.0,0.95,22153.00,8231.00
4025,2019-09-21,36 ST,Sat,11244.0,8457.0,19701.0,0.95,32353.40,12652.40
12005,2019-09-07,CORTELYOU RD,Sat,4358.0,1597.0,5955.0,0.95,9250.00,3295.00
15296,2019-09-22,HOWARD BCH JFK,Sun,3901.0,4269.0,8170.0,0.95,10426.80,2256.80
13478,2019-07-30,FLATBUSH AV-B.C,Tue,18279.0,11053.0,29332.0,0.95,37413.70,8081.70


# End of group data cleaning 

The data should now have outliers removed and can be aggregated as necessary for analysis. The last cell will save a pickled dataframe for easy import into other notebooks.

In [456]:
df_turns_no_outlier.to_pickle('Cleaned_Working_Dataframe.pkl')